In [1]:
# importing libraries
import numpy as np
import pandas as pd
from datasets import load_dataset
import contractions
import re

import unicodedata
import emoji


Hugging face's dataset has the neutral emotions indexed as 0.

In [2]:
dataset_name = "go_emotions"
dataset = load_dataset(dataset_name, "simplified")
pd.set_option('display.max_colwidth', None)

Found cached dataset go_emotions (/Users/heaven/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
len(dataset["train"])

43410

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5427
    })
})

In [5]:
df_train = pd.DataFrame(dataset['train'])
df_val = pd.DataFrame(dataset['validation'])
df_test = pd.DataFrame(dataset['test'])

# df_train = df_train.drop(columns="id", axis=0)
# df_val = df_val.drop(columns="id", axis=0)
# df_test = df_test.drop(columns="id", axis=0)

In [6]:
df_train[:20]

,text,labels,id
0,My favourite food is anything I didn't have to cook myself.,[27],eebbqej
1,"Now if he does off himself, everyone will think hes having a laugh screwing with people instead of actually dead",[27],ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,[2],eezlygj
3,To make her feel threatened,[14],ed7ypvh
4,Dirty Southern Wankers,[3],ed0bdzj
5,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe PlAyOfFs! Dumbass Broncos fans circa December 2015.,[26],edvnz26
6,Yes I heard abt the f bombs! That has to be why. Thanks for your reply:) until then hubby and I will anxiously wait 😝,[15],ee3b6wu
7,We need more boards and to create a bit more space for [NAME]. Then we’ll be good.,"[8, 20]",ef4qmod
8,Damn youtube and outrage drama is super lucrative for reddit,[0],ed8wbdn
9,It might be linked to the trust factor of your friend.,[27],eczgv1o


In [7]:
# emotion_mapping = {
#     '0': 'neutral',
#     '1' : 'admiration',
#     '2' : 'amusement',
#     '3' : 'anger',
#     '4' : 'annoyance',
#     '5' : 'approval',
#     '6' : 'caring',
#     '7' : 'confusion',
#     '8' : 'curiosity',
#     '9' : 'desire',
#     '10': 'disappointment',
#     '11': 'disapproval',
#     '12': 'disgust',
#     '13': 'embarrassment',
#     '14': 'excitement',
#     '15': 'fear',
#     '16': 'gratitude',
#     '17': 'grief',
#     '18': 'joy',
#     '19': 'love',
#     '20': 'nervousness',
#     '21': 'optimism',
#     '22': 'pride',
#     '23': 'realization',
#     '24': 'relief',
#     '25': 'remorse',
#     '26': 'sadness',
#     '27': 'surprise',
    
# }

In [8]:
emotion_mapping = {
    '0': 'admiration',
    '1': 'amusement',
    '2': 'anger',
    '3': 'annoyance',
    '4': 'approval',
    '5': 'caring',
    '6': 'confusion',
    '7': 'curiosity',
    '8': 'desire',
    '9': 'disappointment',
    '10': 'disapproval',
    '11': 'disgust',
    '12': 'embarrassment',
    '13': 'excitement',
    '14': 'fear',
    '15': 'gratitude',
    '16': 'grief',
    '17': 'joy',
    '18': 'love',
    '19': 'nervousness',
    '20': 'optimism',
    '21': 'pride',
    '22': 'realization',
    '23': 'relief',
    '24': 'remorse',
    '25': 'sadness',
    '26': 'surprise',
    '27': 'neutral'
}

# inverting the mapping so that the emotions are now the keys and the integers are the values
inverse_emotion_mapping = {v: int(k) for k, v in emotion_mapping.items()}

In [9]:
def replace_labels_with_text(labels_list):
    return [emotion_mapping[str(label)] for label in labels_list]

df_train['labels'] = df_train['labels'].apply(replace_labels_with_text)
df_val['labels'] = df_val['labels'].apply(replace_labels_with_text)
df_test['labels'] = df_test['labels'].apply(replace_labels_with_text)

In [10]:
df_train[:10]

,text,labels,id
0,My favourite food is anything I didn't have to cook myself.,[neutral],eebbqej
1,"Now if he does off himself, everyone will think hes having a laugh screwing with people instead of actually dead",[neutral],ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,[anger],eezlygj
3,To make her feel threatened,[fear],ed7ypvh
4,Dirty Southern Wankers,[annoyance],ed0bdzj
5,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe PlAyOfFs! Dumbass Broncos fans circa December 2015.,[surprise],edvnz26
6,Yes I heard abt the f bombs! That has to be why. Thanks for your reply:) until then hubby and I will anxiously wait 😝,[gratitude],ee3b6wu
7,We need more boards and to create a bit more space for [NAME]. Then we’ll be good.,"[desire, optimism]",ef4qmod
8,Damn youtube and outrage drama is super lucrative for reddit,[admiration],ed8wbdn
9,It might be linked to the trust factor of your friend.,[neutral],eczgv1o


In [11]:
df_train['text'] = df_train['text'].str.lower()
df_val['text'] = df_val['text'].str.lower()
df_test['text'] = df_test['text'].str.lower()

In [12]:
df_train[:10]

,text,labels,id
0,my favourite food is anything i didn't have to cook myself.,[neutral],eebbqej
1,"now if he does off himself, everyone will think hes having a laugh screwing with people instead of actually dead",[neutral],ed00q6i
2,why the fuck is bayless isoing,[anger],eezlygj
3,to make her feel threatened,[fear],ed7ypvh
4,dirty southern wankers,[annoyance],ed0bdzj
5,omg peyton isn't good enough to help us in the playoffs! dumbass broncos fans circa december 2015.,[surprise],edvnz26
6,yes i heard abt the f bombs! that has to be why. thanks for your reply:) until then hubby and i will anxiously wait 😝,[gratitude],ee3b6wu
7,we need more boards and to create a bit more space for [name]. then we’ll be good.,"[desire, optimism]",ef4qmod
8,damn youtube and outrage drama is super lucrative for reddit,[admiration],ed8wbdn
9,it might be linked to the trust factor of your friend.,[neutral],eczgv1o


## Remove URLs 

In [13]:
def remove_urls_and_html(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    return text

df_train['text'] = df_train['text'].apply(remove_urls_and_html)
df_val['text'] = df_val['text'].apply(remove_urls_and_html)
df_test['text'] = df_test['text'].apply(remove_urls_and_html)

## Remove `r/` in front of the examples

In [14]:
df_train.loc[df_train['text'].str.startswith('r/'), 'text'] = df_train['text'].str.replace(r'^r/', '', regex=True)
df_val.loc[df_val['text'].str.startswith('r/'), 'text'] = df_val['text'].str.replace(r'^r/', '', regex=True)
df_test.loc[df_test['text'].str.startswith('r/'), 'text'] = df_test['text'].str.replace(r'^r/', '', regex=True)

## Appling Contractions

In [15]:
def expand_text(text):
    expanded_words = []    
    for word in text.split():
      expanded_words.append(contractions.fix(word)) 
    return ' '.join(expanded_words)

df_train['text'] = df_train['text'].apply(expand_text)
df_val['text'] = df_val['text'].apply(expand_text)
df_test['text'] = df_test['text'].apply(expand_text)

In [16]:
df_train[:20]

,text,labels,id
0,my favourite food is anything i did not have to cook myself.,[neutral],eebbqej
1,"now if he does off himself, everyone will think hes having a laugh screwing with people instead of actually dead",[neutral],ed00q6i
2,why the fuck is bayless isoing,[anger],eezlygj
3,to make her feel threatened,[fear],ed7ypvh
4,dirty southern wankers,[annoyance],ed0bdzj
5,omg peyton is not good enough to help us in the playoffs! dumbass broncos fans circa december 2015.,[surprise],edvnz26
6,yes i heard about the f bombs! that has to be why. thanks for your reply:) until then hubby and i will anxiously wait 😝,[gratitude],ee3b6wu
7,we need more boards and to create a bit more space for [name]. then we will be good.,"[desire, optimism]",ef4qmod
8,damn youtube and outrage drama is super lucrative for reddit,[admiration],ed8wbdn
9,it might be linked to the trust factor of your friend.,[neutral],eczgv1o


## Remove Punctuations

In [17]:
punctuations_to_remove = r'!"#$%&\'()*+,-./:;<=>?@\\^_`{|}~'

def remove_punctuation(text):
    pattern = rf'(?<!\[)[{re.escape(punctuations_to_remove)}](?![^\[]*\])'
    return re.sub(pattern, '', text)

# Apply the function to the 'text' column in your DataFrame
df_train['text'] = df_train['text'].apply(remove_punctuation)


df_train['text'] = df_train['text'].apply(remove_punctuation)
df_val['text'] = df_val['text'].apply(remove_punctuation)
df_test['text'] = df_test['text'].apply(remove_punctuation)

In [18]:
df_train[:20]

,text,labels,id
0,my favourite food is anything i did not have to cook myself,[neutral],eebbqej
1,now if he does off himself everyone will think hes having a laugh screwing with people instead of actually dead,[neutral],ed00q6i
2,why the fuck is bayless isoing,[anger],eezlygj
3,to make her feel threatened,[fear],ed7ypvh
4,dirty southern wankers,[annoyance],ed0bdzj
5,omg peyton is not good enough to help us in the playoffs dumbass broncos fans circa december 2015,[surprise],edvnz26
6,yes i heard about the f bombs that has to be why thanks for your reply until then hubby and i will anxiously wait 😝,[gratitude],ee3b6wu
7,we need more boards and to create a bit more space for [name] then we will be good,"[desire, optimism]",ef4qmod
8,damn youtube and outrage drama is super lucrative for reddit,[admiration],ed8wbdn
9,it might be linked to the trust factor of your friend,[neutral],eczgv1o


## Remove awkard spaces

In [19]:
def remove_space(text):
    '''Removes awkward spaces from the text.'''
    text = text.strip()
    text = ' '.join(text.split())
    return text

df_train['text'] = df_train['text'].apply(remove_space)
df_val['text'] = df_val['text'].apply(remove_space)
df_test['text'] = df_test['text'].apply(remove_space)

In [20]:
df_train[:20]

,text,labels,id
0,my favourite food is anything i did not have to cook myself,[neutral],eebbqej
1,now if he does off himself everyone will think hes having a laugh screwing with people instead of actually dead,[neutral],ed00q6i
2,why the fuck is bayless isoing,[anger],eezlygj
3,to make her feel threatened,[fear],ed7ypvh
4,dirty southern wankers,[annoyance],ed0bdzj
5,omg peyton is not good enough to help us in the playoffs dumbass broncos fans circa december 2015,[surprise],edvnz26
6,yes i heard about the f bombs that has to be why thanks for your reply until then hubby and i will anxiously wait 😝,[gratitude],ee3b6wu
7,we need more boards and to create a bit more space for [name] then we will be good,"[desire, optimism]",ef4qmod
8,damn youtube and outrage drama is super lucrative for reddit,[admiration],ed8wbdn
9,it might be linked to the trust factor of your friend,[neutral],eczgv1o


In [21]:
label_to_idx = {
    'admiration': '1',
    'amusement': '2',
    'anger': '3',
    'annoyance': '4',
    'approval': '5',
    'caring': '6',
    'confusion': '7',
    'curiosity': '8',
    'desire': '9',
    'disappointment': '10',
    'disapproval': '11',
    'disgust': '12',
    'embarrassment': '13',
    'excitement': '14',
    'fear': '15',
    'gratitude': '16',
    'grief': '17',
    'joy': '18',
    'love': '19',
    'nervousness': '20',
    'optimism': '21',
    'pride': '22',
    'realization': '23',
    'relief': '24',
    'remorse': '25',
    'sadness': '26',
    'surprise': '27',
    'neutral': '28'
}

In [22]:
def labels_to_indices(labels_list):
    return [np.int32(label_to_idx[label]) for label in labels_list if label in label_to_idx]

df_train['labels'] = df_train['labels'].apply(labels_to_indices)
df_val['labels'] = df_val['labels'].apply(labels_to_indices)
df_test['labels'] = df_test['labels'].apply(labels_to_indices)

In [23]:
df_train[:20]

,text,labels,id
0,my favourite food is anything i did not have to cook myself,[28],eebbqej
1,now if he does off himself everyone will think hes having a laugh screwing with people instead of actually dead,[28],ed00q6i
2,why the fuck is bayless isoing,[3],eezlygj
3,to make her feel threatened,[15],ed7ypvh
4,dirty southern wankers,[4],ed0bdzj
5,omg peyton is not good enough to help us in the playoffs dumbass broncos fans circa december 2015,[27],edvnz26
6,yes i heard about the f bombs that has to be why thanks for your reply until then hubby and i will anxiously wait 😝,[16],ee3b6wu
7,we need more boards and to create a bit more space for [name] then we will be good,"[9, 21]",ef4qmod
8,damn youtube and outrage drama is super lucrative for reddit,[1],ed8wbdn
9,it might be linked to the trust factor of your friend,[28],eczgv1o


In [24]:
df_train.to_csv('../goemotions/go_emotions__train_clean.tsv', sep='\t', index=False)
df_val.to_csv('../goemotions/go_emotions__val_clean.tsv',  sep='\t',index=False)
df_test.to_csv('../goemotions/go_emotions_test_clean.tsv',  sep='\t',index=False)

In [25]:
# How to read the file later
df = pd.read_csv("../goemotions/go_emotions__train_clean.tsv", delimiter='\t')

In [26]:
df[:20] 

,text,labels,id
0,my favourite food is anything i did not have to cook myself,[28],eebbqej
1,now if he does off himself everyone will think hes having a laugh screwing with people instead of actually dead,[28],ed00q6i
2,why the fuck is bayless isoing,[3],eezlygj
3,to make her feel threatened,[15],ed7ypvh
4,dirty southern wankers,[4],ed0bdzj
5,omg peyton is not good enough to help us in the playoffs dumbass broncos fans circa december 2015,[27],edvnz26
6,yes i heard about the f bombs that has to be why thanks for your reply until then hubby and i will anxiously wait 😝,[16],ee3b6wu
7,we need more boards and to create a bit more space for [name] then we will be good,"[9, 21]",ef4qmod
8,damn youtube and outrage drama is super lucrative for reddit,[1],ed8wbdn
9,it might be linked to the trust factor of your friend,[28],eczgv1o
